# Training: Personalized Recommendations & Engagement Analytics

In [ ]:
import pandas as pd
users = pd.read_csv('users.csv')
courses = pd.read_csv('courses.csv')
inter = pd.read_csv('course_interactions.csv', parse_dates=['date'])
print(users.head(), courses.head(), inter.head(), sep='\n\n')

## Engagement KPIs

In [ ]:
import matplotlib.pyplot as plt
daily = inter.groupby('date').size().reset_index(name='interactions')
daily.plot(x='date', y='interactions', figsize=(8,3), title='Daily Course Interactions')
plt.show()

## Simple Content-Based Recommender (department & role affinity)

In [ ]:
import numpy as np, pandas as pd
users = pd.read_csv('users.csv')
courses = pd.read_csv('courses.csv')
inter = pd.read_csv('course_interactions.csv')
aff_map = {
    'Finance': ['Finance','Data'],
    'IT': ['IT','AI','Data'],
    'Operations': ['Operations','Safety','AI'],
    'Maintenance': ['Operations','Safety','AI'],
    'HR': ['Safety','Operations','IT'],
    'CustomerService': ['Operations','Data']
}
def recommend_for_user(uid, top_k=5):
    u = users.loc[users.user_id==uid].iloc[0]
    taken = set(inter.loc[inter.user_id==uid, 'course_id'].tolist())
    prefer = set(aff_map.get(u['department'], []))
    def score(row):
        s = 0
        if row['category'] in prefer: s += 2
        if u['tenure_months'] < 24 and row['difficulty']=='Beginner': s += 1
        if 24 <= u['tenure_months'] < 60 and row['difficulty']=='Intermediate': s += 1
        if u['tenure_months'] >= 60 and row['difficulty']=='Advanced': s += 1
        return s
    pool = courses[~courses.course_id.isin(taken)].copy()
    pool['score'] = pool.apply(score, axis=1)
    return pool.sort_values('score', ascending=False).head(top_k)[['course_id','title','category','difficulty','score']]
print(recommend_for_user(3))